In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.set_printoptions(precision=4,    
    sci_mode=False)

In [37]:
pts = torch.randint(0, 9, (1, 3, 2, 2)).float()
pts, pts.shape

(tensor([[[[6., 0.],
           [2., 4.]],
 
          [[2., 4.],
           [5., 7.]],
 
          [[4., 7.],
           [8., 2.]]]]),
 torch.Size([1, 3, 2, 2]))

In [9]:
hdist =  nn.Conv2d(3, 3, 2)
hdist.weight.shape

torch.Size([3, 3, 2, 2])

In [31]:
hood = [[+1., -1.],
        [+0., +0.]]

zeros = [[0.000, 0.000],
         [0.000, 0.000],]

weights = torch.tensor([
    [hood, zeros, zeros],
    [zeros, hood, zeros],
    [zeros, zeros, hood],
])

print(weights.shape)
hdist =  nn.Conv2d(3, 3, 2)
hdist.weight.data = weights
hdist.requires_grad_(False)
hdist(pts), hdist(pts).shape

torch.Size([3, 3, 2, 2])


(tensor([[[[-0.8143]],
 
          [[-0.7561]],
 
          [[-1.0464]]]]),
 torch.Size([1, 3, 1, 1]))

In [33]:
pts

tensor([[[[ 0.,  1.],
          [-1.,  2.]],

         [[ 0.,  1.],
          [-1.,  2.]],

         [[ 0.,  1.],
          [-1.,  2.]]]])

In [27]:
third = 1/3
hood = [[third, third],
        [    0,     0]]

weights = torch.tensor([
    [hood, hood, hood],
])

print(weights.shape)
hdist =  nn.Conv2d(3, 1, 2)
hdist.weight.data = weights
hdist.requires_grad_(False)
hdist(pts).squeeze(), hdist(pts).shape

torch.Size([1, 3, 2, 2])


(tensor(1.1445), torch.Size([1, 1, 1, 1]))

In [40]:
torch.abs(F.conv2d(pts, weights))

tensor([[[[6.]],

         [[2.]],

         [[3.]]]])

In [64]:
def horizontal_distance(pts):
    bs = pts.size(0)
    hood = [[+1., -1.],
            [+0., +0.]]

    zeros = [[0.000, 0.000],
             [0.000, 0.000],]

    weights = torch.tensor([
        [hood, zeros, zeros],
        [zeros, hood, zeros],
        [zeros, zeros, hood],
    ]).to(pts.device)
    dist = F.conv2d(pts, weights) ** 2
    dist = dist.sum(dim=1)    
    dist = torch.sqrt(dist)
    dist = dist.reshape(bs, -1).mean(1)
    return dist

def vertical_distance(pts):
    bs = pts.size(0)
    hood = [[+1., +0.],
            [-1., +0.]]

    zeros = [[+0., +0.],
             [+0., +0.],]

    weights = torch.tensor([
        [hood, zeros, zeros],
        [zeros, hood, zeros],
        [zeros, zeros, hood],
    ]).to(pts.device)
    dist = F.conv2d(pts, weights) ** 2
    dist = dist.sum(dim=1)    
    dist = torch.sqrt(dist)
    dist = dist.reshape(bs, -1).mean(1)
    return dist


    
pts = torch.randint(0, 9, (2, 3, 5, 5)).float()
h_dist = horizontal_distance(pts)
v_dist = vertical_distance(pts)
h_dist, v_dist

(tensor([6.1536, 5.9232]), tensor([6.2034, 5.6097]))

In [66]:
(h_dist + v_dist) / 2

tensor([6.1785, 5.7665])

In [68]:
def mean_distance(pts):    
    def get_distance(pts, hood):
        zeros = [[0., 0.],
                 [0., 0.],]

        weights = torch.tensor([
            [hood, zeros, zeros],
            [zeros, hood, zeros],
            [zeros, zeros, hood],
        ]).to(pts.device)
        dist = F.conv2d(pts, weights) ** 2
        dist = dist.sum(dim=1)    
        dist = torch.sqrt(dist)
        dist = dist.reshape(pts.size(0), -1)
        return dist.mean(1)
    
    v_hood = [[+1., +0.],
              [-1., +0.]]
    h_hood = [[+1., -1.],
              [+0., +0.]]
    v_dist = get_distance(pts, v_hood)
    h_dist = get_distance(pts, h_hood)
    return (v_dist + h_dist) / 2

mean_distance(pts)

tensor([6.1785, 5.7665])

In [71]:
from types import SimpleNamespace

sn = SimpleNamespace(k=[ 128, 128, 256, 256])
sn

namespace(k=[128, 128, 256, 256])

In [70]:
sn.f = 1
sn

namespace(f=1)